# Modelling Rogue Wave Data with Random Forest Regression

In [1]:
%load_ext autoreload
%autoreload 2

## Setup
### Imports

Importing all required packages and define seed and number of cores to use.

In [ ]:
import os
import sys
import pickle
import shap

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV, SequentialFeatureSelector
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import spearmanr

sys.path.append('./')
sys.path.append('../scripts/')
import utils

ModuleNotFoundError: No module named 'utils'

### Parameter Settings

In [ ]:
seed = 42
n_jobs = 4
print(f"Using {n_jobs} cores from {os.cpu_count()} available cores.") # how many CPU cores are available on the current machine

We load the data that was preprocessed in `data_preprocessing.ipynb`.  

In [ ]:
file_data = "../data/data_train_test.pickle"  # path to the preprocessed data
data_train, data_test, y_train, y_train_cat, X_train, y_test, y_test_cat, X_test = utils.load_data(file_data)

## Building a Random Forest Regression Model

### Instantiating the Model and Setting Hyperparameters

- `n_estimators`: The number of trees in the forest.
- `max_depth`: The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
- `max_samples`: If bootstrap is True, the number of samples to draw from X to train each base estimator.
- `criterion`: The function to measure the quality of a split. Supported criteria are “squared_error” for the mean squared error, which is equal to variance reduction as feature selection criterion and minimizes the L2 loss using the mean of each terminal node, “friedman_mse”, which uses mean squared error with Friedman’s improvement score for potential splits, “absolute_error” for the mean absolute error, which minimizes the L1 loss using the median of each terminal node, and “poisson” which uses reduction in Poisson deviance to find splits. Training using “absolute_error” is significantly slower than when using “squared_error”.
- `max_features`: The number of features to consider when looking for the best split.

In [ ]:
num_cv = 5

hyperparameter_grid = {'n_estimators': [100], # not too high to reduce model size
            'max_depth': [20, 30],
            'max_samples': [0.3, 0.4, 0.5], # not too high to reduce model size and since we have many smaples we can afford to set it low
            'criterion': ['friedman_mse'], # using friedman_mse because the target variable is approx. normally distributed but we can expect non-linear relationships or strong feature interactions since the ElNet model has very low performance
            'max_features': ['sqrt'], # leave default since both sqrt and log2 lead to the same number of selected features, i.e. 4 since the total nu,mber of features is small (17)
            'min_samples_leaf': [1, 2, 3, 5]
}

In [ ]:
regressor = RandomForestRegressor(random_state=seed)

### Train the Model

For hyperparameter tuning, we use a k-fold crossvalidation with a stratified splitter that ensures we have enough rogue wave data in the training and validation set.

In [ ]:
# Run CV
model, cv_results = utils.run_CV(regressor, hyperparameter_grid, num_cv, X_train, y_train_cat, y_train)
cv_results.sort_values(by="score", ascending=False)

file_cv = f"../results/random_forest/cv_results.csv"
cv_results.to_csv(file_cv)

In [ ]:
cv_results = pd.read_csv(file_cv)
cv_results

### Evaluate the Model

The CV results above show that the top 3 performing models with crossvalidation score of R^2 > 0.9 are the models with 'max_depth' = 30 and 'max_samples' = 0.5 combined with 'min_samples_leaf' = 1 or 3, or 'max_samples' = 0.3 combined with 'min_samples_leaf' = 1.  
We now check the model size of those top 3 models since the size has a large impact on the ability to run XAI analysis like SHAP.

In [ ]:
hyperparameters_CV = [
    {'max_samples': 0.5, 'min_samples_leaf': 1},
    {'max_samples': 0.4, 'min_samples_leaf': 1},
    {'max_samples': 0.5, 'min_samples_leaf': 2},
]

In [ ]:
for hyperparameters in hyperparameters_CV:
    print("\n")
    # Define the RF regressor
    model = RandomForestRegressor(oob_score=False, random_state=seed, criterion='friedman_mse', max_depth=30, max_features='sqrt', n_estimators=100, max_samples=hyperparameters['max_samples'], min_samples_leaf=hyperparameters['min_samples_leaf'])
    model.fit(X_train, y_train)

    utils.evaluate_best_regressor(model, X_train, y_train, dataset="Training", plot=False)

    file_model_size_test = f'./model_size_test.pickle'
    with open(file_model_size_test, 'wb') as handle:
        pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Get the size of the saved file
    model_size_gb = os.path.getsize(file_model_size_test) / (1024 ** 3)
    print(f"Model size on disk: {model_size_gb:.4f} GB")

    os.remove(file_model_size_test)

We choose to use the model with ```max_samples=0.5``` and ```min_samples_leaf=2``` as this seems to be a good tradeoff between model size and model performance.

In [ ]:
model = RandomForestRegressor(oob_score=False, random_state=seed, criterion='friedman_mse', max_depth=30, max_features='sqrt', n_estimators=100, max_samples=0.5, min_samples_leaf=2)
model.fit(X_train, y_train)

utils.evaluate_best_regressor(model, X_train, y_train, dataset="Training")
utils.evaluate_best_regressor(model, X_test, y_test, dataset="Test")

# Save the model
data_and_model = [data_train, data_test, model]

file_data_model = f"../results/random_forest/model_and_data.pickle"
with open(file_data_model, 'wb') as handle:
    pickle.dump(data_and_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Feature Selection

For the regression model we would like to reduce the number of required features, which would make it easier to use as forecasting model.  
Some of the recoded features are rather hard to measure, e.g. *lambda_30*, *lamda_40*, *s*, *Delta_T*, *nu*, *Q_p*, while other are easier to measure, e.g. *wind*, *Hs*, *p*, *swell*, *kh*, *T_air*, *L_deep*, *T_p*, *Delta_p_1h*.

### Load Data and Model

In [ ]:
model_full, X_train, y_train, y_train_cat, X_test, y_test, y_test_cat = utils.load_data_and_model(file_data_model)

Check the correlation matrix to see if we have redundant features.

In [ ]:
utils.plot_correlation_matrix(X_train, figsize=(5, 5), annot=False, labelsize=8)

### Feature Selection with Recursive Feature Elimination

A Recursive Feature Elimination (RFE) example with automatic tuning of the number of features selected with cross-validation.  
For further information see:
- https://scikit-learn.org/1.5/auto_examples/feature_selection/plot_rfe_with_cross_validation.html
- https://medium.com/@loyfordmwenda/recursive-feature-rfe-elimination-with-scikit-learn-d0d29e96273d

In [ ]:
# Define the regression model and use the hyperparameters from gridsearch for full regression model
model_full, X_train, y_train, y_train_cat, X_test, y_test, y_test_cat = utils.load_data_and_model(file_data_model, output=False)
params = model_full.get_params()

# Initialize a new Random Forest Regressor with the same parameter settings
model = RandomForestRegressor(**params)

# Define splitter
skf = StratifiedKFold(n_splits=num_cv).split(X_train, y_train_cat)

In [ ]:
# Define the RFE parameters
min_features_to_select = 1  # Minimum number of features to consider

In [ ]:
# Run RFE
rfecv = RFECV(
    estimator=model,
    step=1,
    cv=skf,
    scoring="r2",
    min_features_to_select=min_features_to_select,
    n_jobs=n_jobs,
)
rfecv.fit(X_train, y_train)

In [ ]:
# Plot results
cv_results = pd.DataFrame(rfecv.cv_results_)
utils.plot_rfe(cv_results)

In [ ]:
print(f"Optimal number of features: {rfecv.n_features_}")
print(f"Selected features: {X_train.columns[rfecv.support_]}")
print(f"Unselected features: {X_train.columns[rfecv.support_ == False]}")

The sequential feature selector returns 12 features leading to the best score. 
Hence, we will retrain the RF model with only those features.

In [ ]:
top_features = X_train.columns[rfecv.support_]

print(f'Building model for top features: {top_features}.')

model = RandomForestRegressor(**params)
model.fit(X_train[top_features], y_train)

# Train performance
y_pred = model.predict(X_train[top_features])
y_true = y_train

print(f"\nTrain set MSE: {round(mean_squared_error(y_true, y_pred), 3)}")
print(f"Train set R^2: {round(r2_score(y_true, y_pred), 3)}")
print(f"Train set Spearman R: {round(spearmanr(y_true, y_pred).correlation, 3)}")

utils.plot_predictions(y_true=y_true, y_pred=y_pred, textstr = f'$R^2={round(r2_score(y_true, y_pred), 3)}$')

# Test performance
y_pred = model.predict(X_test[top_features])
y_true = y_test

print(f"\nTest set MSE: {round(mean_squared_error(y_true, y_pred), 3)}")
print(f"Test set R^2: {round(r2_score(y_true, y_pred), 3)}")
print(f"Test set Spearman R: {round(spearmanr(y_true, y_pred).correlation, 3)}")

utils.plot_predictions(y_true=y_true, y_pred=y_pred, textstr = f'$R^2={round(r2_score(y_true, y_pred), 3)}$')

### Sequential Feature Selection

SFS is a greedy procedure where, at each iteration, we choose the best new feature to add to our selected features based a cross-validation score. That is, we start with 0 features and choose the best single feature with the highest score. The procedure is repeated until we reach the desired number of selected features.  
For further information see:

- https://scikit-learn.org/1.5/auto_examples/feature_selection/plot_select_from_model_diabetes.html#sphx-glr-auto-examples-feature-selection-plot-select-from-model-diabetes-py


In [ ]:
# Define the regression model and use the hyperparameters from gridsearch for full regression model
model_full, X_train, y_train, y_train_cat, X_test, y_test, y_test_cat = utils.load_data_and_model(file_data_model, output=False)
params = model_full.get_params()

# Initialize a new Random Forest Regressor with the same parameter settings
model = RandomForestRegressor(**params)

# Define splitter
skf = StratifiedKFold(n_splits=num_cv).split(X_train, y_train_cat)

In [ ]:
# Define the SFS parameters
tol = 0.05 # we use the r^2 score for a RandomForest Regressor
n_features_to_select = "auto"
direction = "forward"
scoring = "r2"

In [ ]:
# Run SFS
sfs = SequentialFeatureSelector(model, n_features_to_select=n_features_to_select, tol=tol, direction=direction, scoring=scoring, cv=skf)
sfs.fit(X_train, y_train)

print(f"Optimal number of features: {sfs.n_features_to_select_}")
print(f"Selected features: {X_train.columns[sfs.support_]}")

The sequential feature selector returns 4 features leading to the best score. 
Hence, we will retrain the RF model with only those features.

In [ ]:
top_features = ['H_s', 'lambda_40', 'T_p', 'v_wind']

print(f'Building model for top features: {top_features}.')

model = RandomForestRegressor(**params)
model.fit(X_train[top_features], y_train)

# Train performance
y_pred = model.predict(X_train[top_features])
y_true = y_train

print(f"\nTrain set MSE: {round(mean_squared_error(y_true, y_pred), 3)}")
print(f"Train set R^2: {round(r2_score(y_true, y_pred), 3)}")
print(f"Train set Spearman R: {round(spearmanr(y_true, y_pred).correlation, 3)}")

utils.plot_predictions(y_true=y_true, y_pred=y_pred, textstr = f'$R^2={round(r2_score(y_true, y_pred), 3)}$')

# Test performance
y_pred = model.predict(X_test[top_features])
y_true = y_test

print(f"\nTest set MSE: {round(mean_squared_error(y_true, y_pred), 3)}")
print(f"Test set R^2: {round(r2_score(y_true, y_pred), 3)}")
print(f"Test set Spearman R: {round(spearmanr(y_true, y_pred).correlation, 3)}")

utils.plot_predictions(y_true=y_true, y_pred=y_pred, textstr = f'$R^2={round(r2_score(y_true, y_pred), 3)}$')

## XAI for Random Forest Regression Model

Next, we will use teh SHAP interpretability method for our Random Forest model to understand which features are important for Rogue Wave prediction. 

### Load Data and Model

In [ ]:
model, X_train, y_train, y_train_cat, X_test, y_test, y_test_cat = utils.load_data_and_model(file_data_model, output=True)

### SHAP

With SHAP we get contrastive explanations that compare the prediction with the average prediction. The global interpretations are consistent with the local explanations, since the Shapley values are the “atomic unit” of the global interpretations.

When using TreeExplainer for a Ranfom Forest model, there will be small variations between the average model prediction and the expected value from SHAP. This behaviour is explained as follows in this GitHub thread:

> It is because of how sklearn records the training samples in the tree models it builds. Random forests use a random subsample of the data to train each tree, and it is that random subsample that is used in sklearn to record the leaf sample weights in the model. Since TreeExplainer uses the recorded leaf sample weights to represent the training dataset, it will depend on the random sampling used during training. This will cause small variations like the ones you are seeing.

To get the exact same values, we would have to sample from the whole background dataset for integrating out features via the Independent masker. However, since the dataset is so large, we have to use a smaller background dataset where we expect to see some deviations from the average model prediction.

In addition, when using SHAP to explain a classifiers output, the default value in TreeExplainer for model_output="raw", which explains the raw output of the model. For regression models, "raw" is the standard output. 

In [ ]:
# This part is executed on the server with: python xai_shap.py
# import xai_shap
# file_shap = xai_shap.run_shap(
#     model_type=model_type, 
#     case=case, 
#     undersample_method=undersample_method, 
#     undersample=undersample, 
#     n_samples_0=46000, 
#     n_samples_1=90000, 
#     n_samples_2=14000, 
#     last_batch=-1, 
#     batch_size=1000, 
#     dir_output=f"../results/random_forest/shap/", 
#     n_jobs=n_jobs, 
#     seed=seed
# )

In [ ]:
file_data = f"../results/random_forest/shap/dataset.pkl"
file_shap = f"../results/random_forest/shap/shap.pkl"

# Load and unpack the data
with open(file_data, "rb") as handle:
    X_sample = pickle.load(handle)

# Load and unpack the shap values
with open(file_shap, "rb") as handle:
    shap_values = pickle.load(handle)


# Base value (model expected value)
expected_value = model_full.predict(X_sample).mean()
print(f'SHAP expected value is: {round(expected_value,2)}')

# Create SHAP Explanation
explanation = shap.Explanation(
    values=shap_values,
    base_values=np.full(len(X_sample), expected_value),
    data=X_sample.values,
    feature_names=X_sample.columns.tolist()
)

In [ ]:
shap.plots.bar(explanation, max_display=17)

In [ ]:
shap.plots.beeswarm(explanation, max_display=17)

In [ ]:
utils.plot_shap_dependence(explanation, X_sample)